In [1]:
import warnings
warnings.filterwarnings("ignore")  # "error", "ignore", "always", "default", "module" or "once"

In [2]:
import numpy as np
import pandas as pd
import random
import os
import copy
#from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import precision_score as precision
from sklearn.metrics import recall_score as recall
from sklearn.metrics import f1_score as f1

from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier

import autokeras as ak
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input, ReLU, Dropout
from tensorflow.keras.models import Model

# Original Data

In [3]:
#declear path to your data
drebin_data_path = 'data/drebin.csv'
columns = list(pd.read_csv('data/dataset-features-categories.csv', header = None).iloc[:,0])
# Importing the dataset
Drebin_data = pd.read_csv(drebin_data_path, names = columns)

X = Drebin_data.iloc[:,range(0,Drebin_data.shape[1]-1)].values
y = Drebin_data.iloc[:, -1].values

lb = LabelBinarizer()
y = lb.fit_transform(y)

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y.astype(int), test_size = 0.3, random_state = 0)

# Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Train-Test

In [4]:
classifiers = {"SVM": SVC(kernel = 'linear', degree=3), "MLP": MLPClassifier(random_state=1, max_iter=300), "XGB": XGBClassifier(), "LGBM": LGBMClassifier()} 

for classifier_pair in classifiers.items():
    print("---------------------------")
    print(classifier_pair[0])
    classifier = classifier_pair[1]
    classifier.fit(X_train, y_train)
    # Predicting the Test set results
    y_pred = classifier.predict(X_test)
    #compute accuracy_score
    accuracy = acc(y_test, y_pred)
    print('accuracy', accuracy)
    
print("---------------------------")

---------------------------
SVM
accuracy 0.9796054090002216
---------------------------
MLP
accuracy 0.9895810241631567
---------------------------
XGB
accuracy 0.9904677455109732
---------------------------
LGBM
accuracy 0.9904677455109732
---------------------------


# AutoKeras Model

In [9]:
"""saved_model=keras.models.load_model('structured_data_classifier/best_model', compile=False)
print(saved_model.summary())"""

# Initialize the structured data classifier.
clf = ak.StructuredDataClassifier(
    overwrite=True, max_trials=10
)  # It tries 3 different models.
# Feed the structured data classifier with training data.
clf.fit(
    x=X_train,
    y=y_train,
    epochs=15,
)

# Evaluate the best model with testing data.
print(clf.evaluate(x=X_test, y=y_test))
model = clf.export_model()
model.summary()

Trial 10 Complete [00h 01m 58s]
val_accuracy: 0.9870004653930664

Best val_accuracy So Far: 0.9879634380340576
Total elapsed time: 00h 23m 36s
INFO:tensorflow:Oracle triggered exit
Epoch 1/15
329/329 [==============================] - 6s 6ms/step - loss: 0.1349 - accuracy: 0.9472
Epoch 2/15
329/329 [==============================] - 2s 6ms/step - loss: 0.0566 - accuracy: 0.9810
Epoch 3/15
329/329 [==============================] - 2s 6ms/step - loss: 0.0399 - accuracy: 0.9864
Epoch 4/15
329/329 [==============================] - 2s 5ms/step - loss: 0.0302 - accuracy: 0.9900
Epoch 5/15
329/329 [==============================] - 2s 5ms/step - loss: 0.0239 - accuracy: 0.9923
Epoch 6/15
329/329 [==============================] - 2s 5ms/step - loss: 0.0182 - accuracy: 0.9949
Epoch 7/15
329/329 [==============================] - 2s 5ms/step - loss: 0.0155 - accuracy: 0.9956
Epoch 8/15
329/329 [==============================] - 2s 5ms/step - loss: 0.0130 - accuracy: 0.9964
Epoch 9/15
329/329 

In [5]:
def autoKerasModel(path_best_model):
    saved_model = keras.models.load_model(path_best_model, compile=True)
    input_layer = Input(shape=(215,))
    """x = saved_model.layers[1](input_layer)
    x = saved_model.layers[2](x)
    x = Dense(units=256)(x)
    x = ReLU()(x)
    x = Dense(units=32)(x)
    x = ReLU()(x)
    x = Dense(units=1)(x)
    x = saved_model.layers[-1](x)"""
    x = saved_model.layers[1](input_layer)
    x = saved_model.layers[2](x)
    x = Dense(units=32)(x)
    x = ReLU()(x)
    x = Dense(units=512)(x)
    x = ReLU()(x)
    x = saved_model.layers[-3](x)
    x = Dense(units=1)(x)
    x = saved_model.layers[-1](x)
    new_model = Model(inputs=input_layer, outputs=x)
    return new_model

In [14]:
autokeras_model=autoKerasModel('structured_data_classifier/best_model')
autokeras_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

autokeras_model.fit(X_train, y_train, epochs=15)
results = autokeras_model.evaluate(X_test, y_test)

print("test loss, test acc:", results)
print(autokeras_model.summary())

Epoch 1/15
329/329 [==============================] - 6s 8ms/step - loss: 0.1320 - accuracy: 0.9487
Epoch 2/15
329/329 [==============================] - 3s 9ms/step - loss: 0.0545 - accuracy: 0.9814
Epoch 3/15
329/329 [==============================] - 3s 8ms/step - loss: 0.0406 - accuracy: 0.9864
Epoch 4/15
329/329 [==============================] - 3s 8ms/step - loss: 0.0307 - accuracy: 0.9904
Epoch 5/15
329/329 [==============================] - 3s 8ms/step - loss: 0.0234 - accuracy: 0.9928
Epoch 6/15
329/329 [==============================] - 3s 8ms/step - loss: 0.0192 - accuracy: 0.9947
Epoch 7/15
329/329 [==============================] - 2s 7ms/step - loss: 0.0174 - accuracy: 0.9944
Epoch 8/15
329/329 [==============================] - 2s 7ms/step - loss: 0.0147 - accuracy: 0.9955
Epoch 9/15
329/329 [==============================] - 2s 8ms/step - loss: 0.0151 - accuracy: 0.9949
Epoch 10/15
329/329 [==============================] - 2s 8ms/step - loss: 0.0295 - accuracy: 0.9927

# Ensemble Learning - Majority Voting

In [6]:
def majority_voting(y_preds, y_test):
    assert y_preds.shape[0] == len(y_test), "y_preds's length is: {} while y_test's length is: {}. They should be equal.".format(y_preds.shape[0],len(y_test))
    y_pred_vote = []
    for preds in y_preds:
        if sum(preds) >= 3:
            y_pred_vote.append(1)
        else:
            y_pred_vote.append(0)
    #compute accuracy_score
    accuracy = acc(y_test, y_pred_vote)
    return accuracy, y_pred_vote

In [16]:
y_preds = np.ndarray(shape=(5,len(y_test)))
i=0
for classifier_pair in classifiers.items():
    classifier = classifier_pair[1]
    y_preds[i] = classifier.predict(X_test)
    i += 1
y_preds[i] = np.transpose(autokeras_model.predict(X_test, verbose=0))
y_preds = np.transpose(y_preds)
accuracy, _ = majority_voting(y_preds, y_test)
print('accuracy', accuracy)

accuracy 0.9906894258479273


# Label Flipping

In [7]:
def attack_label_flipping(X_train, X_test, flipped_data, y_test, per, classifier_name, classifier):
    accuracy = 0
    if classifier_name == "AutoKeras":
        classifier.fit(X_train, flipped_data, verbose=0, epochs=15)
        y_pred = np.transpose(classifier.predict(X_test, verbose=0))
        accuracy = classifier.evaluate(X_test, y_test, verbose=0)[1]
    else:
        classifier.fit(X_train, flipped_data)
        y_pred = classifier.predict(X_test)
        accuracy = acc(y_test, y_pred)
    return y_pred, accuracy

In [8]:
#Flipping Random
def random_label_flipping(y_train, per):
    flip_count = int(per*(len(y_train)))
    flipped_data = copy.deepcopy(y_train)
    indices = random.sample(range(len(flipped_data)), flip_count)
    for j in indices:
        flipped_data[j] = (flipped_data[j] + 1)%2
    return flipped_data

#Flipping Specific
def specific_label_flipping(y_train, per, target):
    flipped_data = copy.deepcopy(y_train)
    possible_indices = []
    for i in range(len(y_train)):
        if y_train[i] == target:
            possible_indices.append(i)
    flip_count = int(per*(len(possible_indices)))
    indices = random.sample(possible_indices, flip_count)
    for j in indices:
        flipped_data[j] = (flipped_data[j] + 1)%2
    return flipped_data

In [32]:
percentages = [0.01, 0.05, 0.1, 0.2]
per = percentages[3]
poisoned_accuracies = {"SVM": [], "MLP": [], "XGB": [], "LGBM": [], "AutoKeras": []}
ensemble_accuracies = []

print("---------------------------")
for i in range(5):
    print("Trial #{} is starting...".format(i+1))
    
    poisoned_classifiers = {"SVM": SVC(kernel = 'linear', degree=3), "MLP": MLPClassifier(random_state=1, max_iter=300), "XGB": XGBClassifier(), "LGBM": LGBMClassifier()} 
    poisoned_autokeras_model=autoKerasModel('structured_data_classifier/best_model')
    poisoned_autokeras_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    poisoned_classifiers["AutoKeras"] = poisoned_autokeras_model
    
    flipped_data = random_label_flipping(y_train, per)
    y_preds = np.ndarray(shape=(len(poisoned_classifiers),len(y_test)))
    j=0
    for classifier_pair in poisoned_classifiers.items():
        poisoned_y_pred, poisoned_accuracy = attack_label_flipping(X_train, X_test, flipped_data, y_test, per, classifier_pair[0], classifier_pair[1])
        y_preds[j] = poisoned_y_pred
        j+=1
        poisoned_accuracies[classifier_pair[0]].append(poisoned_accuracy)
    y_preds = np.transpose(y_preds)
    
    ensemble_accuracy, _ = majority_voting(y_preds, y_test)
    ensemble_accuracies.append(ensemble_accuracy)
    
    print("Trial #{} is completed with accuracy: {}".format(i+1, ensemble_accuracy))
    print("---------------------------")

print("Random Poisoning - {}".format(per))
for classifier_pair in poisoned_accuracies.items():
    accuracies = classifier_pair[1]
    print("{}'s average accuracy: {}".format(classifier_pair[0], sum(accuracies)/len(accuracies)))
print("Average ensemble accuracy:", sum(ensemble_accuracies)/len(ensemble_accuracies))

---------------------------
Trial #1 is starting...
Trial #1 is completed with accuracy: 0.97251163821769
---------------------------
Trial #2 is starting...
Trial #2 is completed with accuracy: 0.97628020394591
---------------------------
Trial #3 is starting...
Trial #3 is completed with accuracy: 0.9711815561959655
---------------------------
Trial #4 is starting...
Trial #4 is completed with accuracy: 0.9729549988915983
---------------------------
Trial #5 is starting...
Trial #5 is completed with accuracy: 0.9771669252937264
---------------------------
Random Poisoning - 0.2
SVM's average accuracy: 0.9626246951895366
MLP's average accuracy: 0.9632454001330082
XGB's average accuracy: 0.9724673021502992
LGBM's average accuracy: 0.976989581024163
AutoKeras's average accuracy: 0.9781866669654846
Average ensemble accuracy: 0.9740190645089781


In [35]:
percentages = [0.01, 0.05, 0.1, 0.2]
per = percentages[3]
poisoned_accuracies = {"SVM": [], "MLP": [], "XGB": [], "LGBM": [], "AutoKeras": []}

poisoned_classifiers = {"SVM": SVC(kernel = 'linear', degree=3), "MLP": MLPClassifier(random_state=1, max_iter=300), "XGB": XGBClassifier(), "LGBM": LGBMClassifier()} 
poisoned_autokeras_model=autoKerasModel('structured_data_classifier/best_model')
poisoned_autokeras_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
poisoned_classifiers["AutoKeras"] = poisoned_autokeras_model

flipped_data = random_label_flipping(y_train, per)
y_preds = np.ndarray(shape=(len(poisoned_classifiers),len(y_test)))
j=0

for classifier_pair in poisoned_classifiers.items():
    poisoned_y_pred, poisoned_accuracy = attack_label_flipping(X_train, X_test, y_train, y_test, per, classifier_pair[0], classifier_pair[1])
    y_preds[j] = poisoned_y_pred
    j+=1
    poisoned_accuracies[classifier_pair[0]].append(poisoned_accuracy)
y_preds = np.transpose(y_preds)
print(poisoned_accuracies)
ensemble_accuracy, y_pred_vote = majority_voting(y_preds, y_test)

print("Trial is completed with accuracy: {}".format(ensemble_accuracy))
print("---------------------------")

y_preds_train = np.ndarray(shape=(len(poisoned_classifiers),len(y_train)))
z=0
for classifier_pair in poisoned_classifiers.items():
    y_pred = []
    if classifier_pair[0] == "AutoKeras":
        y_pred = np.transpose(classifier_pair[1].predict(X_train, verbose=0))
        accuracy = classifier_pair[1].evaluate(X_train, y_train)[1]
    else:
        y_pred = classifier_pair[1].predict(X_train)
        accuracy = acc(y_train, y_pred)
    y_preds_train[z] = y_pred
    z+=1
    print(classifier_pair[0], accuracy)

y_preds_train = np.transpose(y_preds_train)
train_acc, y_pred_train_vote = majority_voting(y_preds_train, y_train)
print("ensemble", train_acc)

{'SVM': [0.9796054090002216], 'MLP': [0.9895810241631567], 'XGB': [0.9904677455109732], 'LGBM': [0.9904677455109732], 'AutoKeras': [0.9842606782913208]}
Trial is completed with accuracy: 0.9909111061848814
---------------------------
SVM 0.985938242280285
MLP 0.9989548693586698
XGB 0.9980997624703087
LGBM 0.9964845605700713
329/329 [==============================] - 3s 9ms/step - loss: 0.0067 - accuracy: 0.9988
AutoKeras 0.9987648725509644
ensemble 0.9970546318289786


In [36]:
t=0
defo_poison = 0
for a in range(len(y_train)):
    if y_train[a] != flipped_data[a]:
        defo_poison +=1
print(defo_poison/len(y_train))
for classifier_pair in poisoned_classifiers.items():
    poison_maybe = 0
    for k in range(len(y_pred_train_vote)):
        if y_pred_train_vote[k] != y_preds_train[k][t]:
            poison_maybe += 1
    t+=1
    print(classifier_pair[0], poison_maybe/len(y_pred_train_vote))
    
#her seferinde data farklı olucak average prediction ı da alamam?
#ya da bu outputların ortalamasını alıcam ama çok farketmez sanki

0.2
SVM 0.011306413301662708
MLP 0.0019002375296912114
XGB 0.0010451306413301663
LGBM 0.0009501187648456057
AutoKeras 0.7222802850356295


# SECML Poisoning

In [4]:
from secml.ml.classifiers.sklearn.c_classifier_logistic import CClassifierLogistic
from secml.adv.attacks.poisoning.c_attack_poisoning_logistic_regression import CAttackPoisoningLogisticRegression
from secml.data.c_dataset import CDataset
from secml.ml.peval.metrics import CMetricAccuracy

lob, ub = 0., 1.  # Bounds of the attack space. Can be set to `None` for unbounded

# Should be chosen depending on the optimization problem
solver_params = {
    'eta': 0.25,
    'eta_min': 2.0,
    'eta_max': None,
    'max_iter': 100,
    'eps': 1e-6
}

dataset = CDataset(X_train, y_train)
metric = CMetricAccuracy()

# train SVM in the dual space, on a linear kernel, as needed for poisoning
c_classifier_LR = CClassifierLogistic()
print("Training of c_classifier_LR...")
c_classifier_LR.fit(X_train, y_train)
print("Training of c_classifier_LR is done!")

Training of c_classifier_LR...
Training of c_classifier_LR is done!


In [5]:
poisoning_strength = 0.1 # Percentage of points to poison in training data
n_poisoning_points = 2  # Number of poisoning points to generate

pois_attack = CAttackPoisoningLogisticRegression(classifier=c_classifier_LR,
                                training_data=dataset,
                                val=dataset,
                                lb=lob, 
                                ub=ub,
                                solver_params=solver_params)
pois_attack.n_points = n_poisoning_points

# Run the poisoning attack
print("Attack started...")
pois_y_pred, _, pois_points_ds, _ = pois_attack.run(X_test, y_test)
print("Attack complete!")

Attack started...
Attack complete!


# Reading Poisoned data & adding to/replacing with training dataset

In [20]:
#declear path to your data
pois_points_path = 'poisoning-20.csv'
# Importing the dataset
pois_points_ds = pd.read_csv(pois_points_path)
pois_points_X = pois_points_ds.iloc[:,range(0,pois_points_ds.shape[1]-1)].values
pois_points_y = pois_points_ds.iloc[:, -1].values

pois_points_y = lb.fit_transform(pois_points_y)

In [21]:
poisoned_accuracies = {"SVM": [], "MLP": [], "XGB": [], "LGBM": [], "AutoKeras": []}
ensemble_accuracies = []
print("---------------------------")
for trial in range(5):
    pois_X_train = copy.deepcopy(X_train)
    pois_y_train = copy.deepcopy(y_train)

    #Uncomment below to RANDOMLY REPLACE with poisoned data
    to_be_replaced = random.sample(range(len(pois_X_train)), len(pois_points_X))
    new_data = 0
    for index in to_be_replaced:
        pois_X_train[index] = pois_points_X[new_data]
        pois_y_train[index] = pois_points_y[new_data]
        new_data += 1

    #Uncomment below to ADD poisoned data
    """pois_X_train = np.append(pois_X_train, pois_points_X, axis=0)
    pois_y_train = np.append(pois_y_train, pois_points_y)"""

    print("Trial #{} is starting...".format(trial+1))
    
    classifiers_secml = {"SVM": SVC(kernel = 'linear', degree=3), "MLP": MLPClassifier(random_state=1, max_iter=300), "XGB": XGBClassifier(), "LGBM": LGBMClassifier()} 
    
    y_preds = np.ndarray(shape=(5,len(y_test)))
    j=0
    for classifier_pair in classifiers_secml.items():
        classifier = classifier_pair[1]
        classifier.fit(pois_X_train, pois_y_train)
        # Predicting the Test set results
        poisoned_y_pred = classifier.predict(X_test)
        y_preds[j] = poisoned_y_pred
        #compute accuracy_score
        poisoned_accuracy = acc(y_test, poisoned_y_pred)
        poisoned_accuracies[classifier_pair[0]].append(poisoned_accuracy)
        j+=1
        
    ak_model_secml=autoKerasModel('structured_data_classifier/best_model')
    ak_model_secml.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    ak_model_secml.fit(pois_X_train, pois_y_train, epochs=15, verbose=0)
    y_preds[j] = np.transpose(ak_model_secml.predict(X_test, verbose=0))
    poisoned_accuracies["AutoKeras"].append(ak_model_secml.evaluate(X_test, y_test, verbose=0)[1])

    y_preds = np.transpose(y_preds)
    
    ensemble_accuracy = majority_voting(y_preds, y_test)
    ensemble_accuracies.append(ensemble_accuracy)
    
    print("Trial #{} is completed with accuracy: {}".format(trial+1, ensemble_accuracy))
    print("---------------------------")

print("SECML Poisoning - 20%")
for classifier_pair in poisoned_accuracies.items():
    accuracies = classifier_pair[1]
    print("{}'s average accuracy: {}".format(classifier_pair[0], sum(accuracies)/len(accuracies)))
print("Average ensemble accuracy:", sum(ensemble_accuracies)/len(ensemble_accuracies))

---------------------------
Trial #1 is starting...
Trial #1 is completed with accuracy: 0.9893593438262026
---------------------------
SECML Poisoning - 10%
SVM's average accuracy: 0.9687430724894702
MLP's average accuracy: 0.9824872533806251
XGB's average accuracy: 0.9886943028153403
LGBM's average accuracy: 0.9895810241631567
AutoKeras's average accuracy: 0.9895810484886169
Average ensemble accuracy: 0.9893593438262026
